In [ ]:
try:
    sc.stop()
except:
    pass

In [174]:
import json

import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [175]:
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as sf

In [176]:
spark

## Films

In [177]:
films = spark.read.option("delimiter", "|").csv('/labs/laba01/ml-100k/u.item')

In [178]:
films.limit(5).toPandas()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,...,_c14,_c15,_c16,_c17,_c18,_c19,_c20,_c21,_c22,_c23
0,1,Toy Story (1995),01-Jan-1995,None,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,None,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,None,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,None,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,None,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


## Ratings

In [213]:
ratings = spark.read.option("delimiter", "\t").csv('/labs/laba01/ml-100k/u.data')

In [214]:
ratings.limit(5).toPandas()

,_c0,_c1,_c2,_c3
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


#### Show the number of ratings for all films

In [215]:
value_counts = ratings.groupBy('_c2').count().orderBy('_c2')

In [216]:
value_counts.limit(5).toPandas()

,_c2,count
0,1,6110
1,2,11370
2,3,27145
3,4,34174
4,5,21201


In [217]:
value_counts = value_counts.withColumnRenamed("count","hist_all")

In [218]:
value_counts = value_counts.toPandas()

#### Show the number of ratings for the film number 302

In [327]:
ratings_302 = ratings.filter(ratings._c1 == 302)

In [328]:
ratings_302.limit(5).toPandas()

,_c0,_c1,_c2,_c3
0,186,302,3,891717742
1,191,302,4,891560253
2,49,302,4,888065432
3,54,302,4,880928519
4,62,302,3,879371909


In [329]:
value_counts_302 = ratings_302.groupBy('_c2').count().orderBy('_c2')

In [331]:
value_counts_302 = value_counts_302.withColumnRenamed("count","hist_film")

In [330]:
value_counts_302.limit(5).toPandas()

,_c2,count
0,1,2
1,2,10
2,3,46
3,4,119
4,5,120


In [332]:
value_counts_302 = value_counts_302.toPandas()

## Join all

In [333]:
value_counts_final = value_counts_302.merge(value_counts, how='inner', on='_c2')
value_counts_final = value_counts_final[['hist_film', 'hist_all']]

value_counts_final_dict = value_counts_final.to_dict(orient='list')

In [336]:
value_counts_final_dict

{'hist_film': [2, 10, 46, 119, 120],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

In [337]:
json_object = json.dumps(value_counts_final_dict, indent = 4) 

In [338]:
print(json_object)

{
    "hist_film": [
        2,
        10,
        46,
        119,
        120
    ],
    "hist_all": [
        6110,
        11370,
        27145,
        34174,
        21201
    ]
}


In [339]:
with open("lab01.json", "w") as outfile:
    json.dump(value_counts_final_dict, outfile)

In [340]:
sc.stop()